<font color='blue'> **Numerical Layers** </font> 

In [1]:
import os; import numpy as np ; import pandas as pd ; import rasterio  ; from rasterio.plot import show  ; from pathlib import Path ; import openpyxl
import matplotlib.pyplot as plt              
TA=Path("./TA") ;   PP = Path('E:/15_REPOS/05_Working/01_nerversink')  ; PB=Path(PP/'PB')   ; PE=Path(PB/'Layers_and_Zones')                                         

<font color='orange'> **Plot** </font> 

In [2]:
with rasterio.open(os.path.join(PE/ 'L3_Bottom___TopOfBedrock.tif')) as src:
    crs=src.crs    ;meta=src.meta   ;nodata=meta['nodata']      ;L3T=src.read(1, masked=True),       ;L3D=L3T[0].data   ;L3M=L3T[0].mask     #;show(src)

In [3]:
L4D = np.where(L3D != nodata, L3D-33, L3D)      
dataset = rasterio.open(os.path.join(TA, 'L4_Bottom.tif'),'w', driver=meta['driver'],height = L4D.shape[0], width = L4D.shape[1],count=1,nodata = nodata,dtype=str(L4D.dtype),
                        crs = crs,transform=meta['transform'],compress='lzw')                                                      ; dataset.write(L4D, 1)   ; dataset.close()

In [4]:
# I0  = np.ma.MaskedArray(L4D, L3M)  ; plt.imshow(I0) ; plt.colorbar()   ; plt.title('Layer 4 - Model Bottom') ; plt.show() # ______ Plot
#L3L4 = L3D-L4D  ;mL3L4=np.ma.MaskedArray(L3L4, L3M)  ; plt.imshow(L3L4) ; plt.colorbar() ; plt.tight_layout() ; plt.show() ; print('dif: {}'.format(list(np.unique(L3L4)))) # ______ Plot

<font color='blue'> **Check the minimum thicknesses of the model layers** </font> 

In [ ]:
TLO = os.path.join(PB, 'top_50m_from_lidar.tif')   ; TL1 = os.path.join(PE, 'L1_Bottom.tif')              ; TL2 = os.path.join(PE, 'L2_Bottom.tif')
with rasterio.open(TLO) as src:
    L0T = src.read(1, masked=True),                ; L0D = L0T[0].data        ; L0M = L0T[0].mask         ; L0ME = src.meta   #; show(src)         # L0ME
with rasterio.open(TL1) as src:
    L1T = src.read(1, masked=True),                ; L1D = L1T[0].data        ; L1M = L1T[0].mask         ; L1ME = src.meta                         # L1ME
with rasterio.open(TL2) as src:
    L2T = src.read(1, masked=True),                ; L2D = L2T[0].data        ; L2M = L2T[0].mask

In [ ]:
L1L2  = L1D - L2D   ; ML1L2 = np.ma.MaskedArray(L1L2, L1M)      # plt.figure(figsize=(10,10)) ; plt.imshow(ML1L2) ; plt.colorbar()  ; plt.show()
L2L3  = L2D - L3D   ; ML2L3 = np.ma.MaskedArray(L2L3, L1M)      # plt.figure(figsize=(10,10)) ; plt.imshow(ML2L3) ; plt.colorbar() ; plt.show()
print('the minimum difference is: {:.05f} meters'.format(ML1L2.min()))  ;print('the minimum difference is: {:.05f} meters'.format(ML2L3.min()))

<font color='orange'> **Plot ________________** </font> 

In [ ]:
L1L2ISO = np.where((L1L2 == 1.), 1, np.nan)                  # plt.figure(figsize=(10,10)) ; plt.imshow(L1L2ISO)   ; plt.title(      'L1L2 = 1.0 meter')  ; plt.show()
L2L3ISO = np.where((ML2L3 == 1.), 1, np.nan)                 # plt.figure(figsize=(10,10)) ; plt.imshow(L1L2ISO)   ; plt.title(      'L2L3 = 1.0 meter')  ; plt.show()
L1L2LESS1 = np.where((L1L2 < 1.) & (L1L2 > 0.), 1, np.nan)   # plt.figure(figsize=(10,10)) ; plt.imshow(L1L2LESS1) ; plt.title('0.0 > L1L2 < 1.0 meters') ; plt.show()

-   Top of model ('top_50m_from_lidar.tif') has different dimensions and cell size than L1, L2, L3, and L4 bottoms <br>
-   MIM  DIFF bottoms of layer 1-2 and layer2-3 appears to be 1., but there are a few scattered points in the raster with differences slightly smaller, equal to 0.99997 meters